### Installs

In [ ]:
!pip install vaderSentiment
!pip install openpyxl
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 17.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import datetime

# For model training
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from setfit import SetFitModel, Trainer, TrainingArguments
from datasets import Dataset

# For model evaluation
from sklearn.metrics import (
    classification_report, confusion_matrix,
    mean_squared_error, mean_absolute_error, r2_score,
    roc_curve, auc, precision_recall_curve, roc_auc_score
)

# For visualization
from seaborn import heatmap
from matplotlib import pyplot as plt

# For export
import openpyxl
import xlsxwriter

import torch
import pickle

pd.set_option('display.max_columns', None)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Functions definitions

In [ ]:
def get_distributions(data, cols):
  plt.close()
  for var in cols:
    fig, ax = plt.subplots()

    if data[var].dtypes == "object":

      # Get value counts for the categorical variable
      counts = data[var].value_counts().sort_index()

      # Create bar plot
      ax.bar(counts.index.astype(str), counts.values, edgecolor='black')
      # Add gridlines
      ax.grid(True, axis='y', alpha=0.3, linestyle='--', linewidth=0.7)
      ax.set_axisbelow(True)

      # Formatting
      ax.set_xlabel(var, fontsize=12)
      ax.set_ylabel('Frequency', fontsize=12)
      ax.set_title(f'Distribution of {var}', fontsize=14)
      plt.xticks(rotation=45, ha='right')

      plt.tight_layout()
      plt.show()
      plt.close()

    else:
      counts, bin_edges, _ = plt.hist(data[var], edgecolor='black')
      plt.xticks(bin_edges, rotation=45)
      plt.xlabel(var)
      plt.ylabel('Frequency')
      plt.fill()
      plt.show()
      plt.close()

In [ ]:
def get_metrics_table(y_true, pred_dict):
  full_metrics_table = pd.DataFrame(columns=['model_output', 'metrics',	'negative',	'neutral',	'positive',	'accuracy',	'macro avg', 'weighted avg'])

  for var_name, y_pred in pred_dict.items():
    curr = pd.DataFrame(classification_report(y_true, pred_dict[var_name], output_dict=True)).reset_index().rename(columns={'index':'metrics'})
    curr['model_output'] = var_name
    full_metrics_table = pd.concat([full_metrics_table, curr])[full_metrics_table.columns]
  full_metrics_table.sort_values(['accuracy'], ascending=True, inplace=True)
  return full_metrics_table

In [ ]:
# For model evaluation - plotting confusion matrix with seaborn
def plot_confusion_matrix(y_true, y_pred, title, xticks = None, yticks= None, color="viridis",  annotation=True):
    plt.close()
    if xticks == None:
      xticks = True
    if yticks == None:
      yticks = True
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots()
    ax = heatmap(cm, annot=annotation, cmap=color,xticklabels=xticks, yticklabels=yticks, fmt='g')
    ax.set(xlabel="Predicted Value", ylabel="True Value")
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.set_title(title)
    plt.close()
    return fig

### Data import

In [ ]:
# Navigate to your data directory
# data_dir = '/content/drive/MyDrive/Colab Notebooks/DermaLLM/data/'
data_dir = '/content/drive/MyDrive/Colab Notebooks/DermaLLM_old_backup/DermaLLM/data/'

# Set directory for plots dump
plot_path = '/content/drive/MyDrive/Colab Notebooks/DermaLLM_old_backup/DermaLLM/data/plots/'

td_date = datetime.datetime.now().strftime("%m_%d_%Y")

In [ ]:
# Import clean reviews table
reviews_df_clean = pd.read_pickle(data_dir + "reviews_data_clean_12_11_2025.pkl")
print(reviews_df_clean.columns)
reviews_df_clean

Index(['author_id', 'rating', 'is_recommended', 'submission_time',
       'review_text', 'review_title', 'skin_tone', 'eye_color', 'skin_type',
       'hair_color', 'product_id', 'product_name', 'brand_name', 'price_usd',
       'helpfulness', 'total_feedback_count', 'total_pos_feedback_count',
       'total_neg_feedback_count', 'unq_review_cnt_per_prod'],
      dtype='object')


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,author_id,rating,is_recommended,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,helpfulness,total_feedback_count,total_pos_feedback_count,total_neg_feedback_count,unq_review_cnt_per_prod
0,538863,1,0.0,2018-11-01,One use and into the trash this went. I woke u...,one and done,fair,blue,combination,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,NaN,0,0,0,10226
1,561736,5,1.0,2018-07-28,This is my nightly hero. It is the one facial...,Awesome!,light,blue,combination,blonde,P421998,Midnight Recovery Concentrate Moisturizing Fac...,Kiehl's Since 1851,56.0,0.833333,6,5,1,729
2,561736,5,1.0,2018-07-28,This is my nightly hero. It is the one facial...,Awesome!,light,blue,combination,blonde,P445951,Midnight Recovery Concentrate Moisturizing Fac...,Kiehl's Since 1851,30.0,0.833333,6,5,1,729
3,602980,5,1.0,2018-10-03,This has become a must have addition to my mor...,Brain Must!!,lightMedium,blue,combination,blonde,P423159,Brain Dust,Moon Juice,38.0,1.000000,1,1,0,37
4,696309,5,1.0,2019-06-17,"Well, I was a skeptic, but after trying a samp...",replacing my other serums,lightMedium,brown,dry,brown,P444222,Luxury Beauty Serum Calming Treatment,Saint Jane Beauty,125.0,1.000000,30,30,0,247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581780,orderGen270100,5,1.0,2019-02-28,I have dry sensitive combination skin and my f...,Can’t beat the price and you can see the resul...,fair,blue,combination,blonde,P427419,Hyaluronic Acid 2% + B5 Hydrating Serum,The Ordinary,15.7,NaN,0,0,0,2351
581781,orderGen39837,5,1.0,2018-05-18,This is great for fine lines! I’ve never revie...,Actually working on my fine lines!,light,green,normal,blonde,P429515,C-Tango Vitamin C Eye Cream,Drunk Elephant,64.0,0.250000,8,2,6,917
581782,orderGen39837,5,1.0,2020-08-24,I got a facial and was told I have beautiful s...,Love this product,light,green,normal,blonde,P456418,Wild Rose Night-Brightening Sleeping Facial,KORRES,50.0,0.900000,10,9,1,222
581783,orderGen51156,5,1.0,2020-02-10,Nice cooling sensation once applied to the ski...,cooling and moisturizing,lightMedium,brown,combination,black,P433443,Aqua Bomb Sleeping Mask,belif,38.0,0.000000,1,0,1,627


### Feature engineering

In [ ]:
reviews_df_clean['review_txt_tt'] = reviews_df_clean['review_text'].astype(str) + " — " + reviews_df_clean['review_title'].astype(str)
print(reviews_df_clean['review_txt_tt'].notnull().mean())

1.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Creating Target

In [ ]:
# Create labels from ratings
# The idea is that if there is a high rating, review should be positive.
reviews_df_clean['sentiment_rating'] = reviews_df_clean['rating'].apply(
    lambda x: 'positive' if x >= 4 else ('negative' if x <= 2 else 'neutral')
)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Check the unique values and their distribution
print("Unique values in sentiment_rating:")
print(reviews_df_clean['sentiment_rating'].unique())
print("\nValue counts:")
print(reviews_df_clean['sentiment_rating'].value_counts())
print("\nData type:")
print(reviews_df_clean['sentiment_rating'].dtype)

Unique values in sentiment_rating:
['negative' 'positive' 'neutral']

Value counts:
sentiment_rating
positive    474060
negative     64395
neutral      43330
Name: count, dtype: int64

Data type:
object


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Set up dictionary for models prediction
# Will be used to get comprehensive metrics tables
pred_dict = {}
cm_dict = {}

data_split_summary = pd.DataFrame(columns=[
    'model_type',
    'text_field',
    'X_train_shape',
    'X_test_shape',
    'y_train_shape',
    'y_test_shape',
    'train_size',
    'test_size',
    'train_positive_ratio',
    'train_neutral_ratio',
    'train_negative_ratio',
    'test_positive_ratio',
    'test_neutral_ratio',
    'test_negative_ratio',
    'random_state'
])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Models Training and Evaluations

We will be using the abbreviations below for new variables:

- "ssc" = "sentiment_score"
- "scp" = "sentiment_compound"
- "slb" = "sentiment_label"
- "rv" = "review"
- "tt" = "title"
- "txt" = "text"
- "neg" = "negative"
- "neu" = "neutral"
- "pos" = "positive"



### 1) Vader Sentiment Analyzer

In [ ]:
vader_results = {}
vader_metrics = None

In [ ]:
row_index = 0

X_vars_dict = {"review_text":"txt", "review_title":"tt", "review_txt_tt":"txt_tt"}

for var, code in X_vars_dict.items():

  # Split data
  X = reviews_df_clean[var]
  y = reviews_df_clean['sentiment_rating']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=62)

  train_pos_ratio = (y_train == 'positive').sum() / len(y_train)
  test_pos_ratio = (y_test == 'positive').sum() / len(y_test)

  train_neu_ratio = (y_train == 'neutral').sum() / len(y_train)
  test_neu_ratio = (y_test == 'neutral').sum() / len(y_test)

  train_neg_ratio = (y_train == 'negative').sum() / len(y_train)
  test_neg_ratio = (y_test == 'negative').sum() / len(y_test)

  data_split_summary.loc[row_index] = {
        'model_type': 'VADER',
        'text_field': var,
        'X_train_shape': X_train.shape,
        'X_test_shape': X_test.shape,
        'y_train_shape': y_train.shape,
        'y_test_shape': y_test.shape,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'train_positive_ratio': f"{train_pos_ratio:.3f}",
        'train_neutral_ratio': f"{train_neu_ratio:.3f}",
        'train_negative_ratio': f"{train_neg_ratio:.3f}",
        'test_positive_ratio': f"{test_pos_ratio:.3f}",
        'test_neutral_ratio': f"{test_neu_ratio:.3f}",
        'test_negative_ratio': f"{test_neg_ratio:.3f}",
        'random_state': 62
    }

  row_index += 1

  analyzer = SentimentIntensityAnalyzer()

  # Apply to each review
  vader_scores = X_test.apply(
      lambda x: analyzer.polarity_scores(str(x))
  )

  # Extract compound score (-1 to 1)
  vader_compound = vader_scores.apply(
      lambda x: x['compound']
  )

  # Create positive/negative label
  vader_labels = vader_compound.apply(
      lambda x: 'positive' if x >= 0.05 else ('negative' if x <= -0.05 else 'neutral')
  )

  # Store predictions
  pred_dict[f'slb_rv_{code}_vader'] = vader_labels
  vader_results[code+'_vader'] = {
      'y_pred': vader_labels,
      'compound_scores': vader_compound,
      'y_test': y_test,
      'X_test': X_test
  }

  cm_dict[var] = plot_confusion_matrix(vader_results[code+'_vader']['y_test'], vader_results[code+'_vader']['y_pred'], xticks=['Negative', 'Neutral', 'Positive'],yticks=['Negative', 'Neutral', 'Positive'], title= var + " - Confusion Matrix")

  vader_metrics = get_metrics_table(vader_results[code+'_vader']['y_test'], pred_dict)
  vader_metrics.sort_values(['accuracy'], ascending=False, inplace=True)



/tmp/ipython-input-2451492381.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_metrics_table = pd.concat([full_metrics_table, curr])[full_metrics_table.columns]
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/tmp/ipython-input-2451492381.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the ol

In [ ]:
# Output results to excel for easier review

with pd.ExcelWriter(data_dir+"vader_models_results_test_"+td_date+".xlsx") as writer:
    data_split_summary.to_excel(writer, sheet_name = "train_test_split", index=False)
    vader_metrics.to_excel(writer, sheet_name = "metrics", index=False)

    workbook = writer.book
    worksheet = writer.sheets['metrics']

    row = 0

    # Add confusion matrices
    for plot_name, fig in cm_dict.items():
          # Set the figure size
          fig.set_size_inches(8, 6)
          # Save the figure to a temporary file
          img_path = plot_path + plot_name + '.png'
          fig.savefig(img_path, bbox_inches='tight')
          # Insert the image into the worksheet
          worksheet.insert_image(row, 10, img_path)
          plt.close(fig)  # Close the figure to free memory
          row += 30

writer.close()

/usr/local/lib/python3.12/dist-packages/xlsxwriter/workbook.py:404: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### 2) TF-IDF Custom Analyzer

In [ ]:
# Set up dictionary for models prediction
# Will be used to get comprehensive metrics tables
pred_dict = {}
cm_dict = {}

data_split_summary = pd.DataFrame(columns=[
    'model_type',
    'text_field',
    'X_train_shape',
    'X_test_shape',
    'y_train_shape',
    'y_test_shape',
    'train_size',
    'test_size',
    'train_positive_ratio',
    'train_neutral_ratio',
    'train_negative_ratio',
    'test_positive_ratio',
    'test_neutral_ratio',
    'test_negative_ratio',
    'random_state'
])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
custom_results = {}
custom_metrics = None

In [ ]:
row_index = 0

X_vars_dict = {"review_text":"txt", "review_title":"tt", "review_txt_tt":"txt_tt"}

for var, code in X_vars_dict.items():

  # Split data
  X = reviews_df_clean[var]
  y = reviews_df_clean['sentiment_rating']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=62)

  train_pos_ratio = (y_train == 'positive').sum() / len(y_train)
  test_pos_ratio = (y_test == 'positive').sum() / len(y_test)

  train_neu_ratio = (y_train == 'neutral').sum() / len(y_train)
  test_neu_ratio = (y_test == 'neutral').sum() / len(y_test)

  train_neg_ratio = (y_train == 'negative').sum() / len(y_train)
  test_neg_ratio = (y_test == 'negative').sum() / len(y_test)

  data_split_summary.loc[row_index] = {
        'model_type': 'Custom Analyzer',
        'text_field': var,
        'X_train_shape': X_train.shape,
        'X_test_shape': X_test.shape,
        'y_train_shape': y_train.shape,
        'y_test_shape': y_test.shape,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'train_positive_ratio': f"{train_pos_ratio:.3f}",
        'train_neutral_ratio': f"{train_neu_ratio:.3f}",
        'train_negative_ratio': f"{train_neg_ratio:.3f}",
        'test_positive_ratio': f"{test_pos_ratio:.3f}",
        'test_neutral_ratio': f"{test_neu_ratio:.3f}",
        'test_negative_ratio': f"{test_neg_ratio:.3f}",
        'random_state': 62
    }
  row_index += 1

  vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=5, max_df=0.7 )
  X_train_tfidf = vectorizer.fit_transform(X_train)
  X_test_tfidf = vectorizer.transform(X_test)

  # Train logistic regression to get probability scores
  lr = LogisticRegression(max_iter=100, random_state=62, class_weight='balanced', solver='saga', n_jobs=-1)
  lr.fit(X_train_tfidf, y_train)

  # Get all probabilities (returns array with shape [n_samples, 3])
  all_probs = lr.predict_proba(X_test_tfidf)

  # Check what the classes are
  print("Class order:", lr.classes_)  # e.g., ['negative', 'neutral', 'positive']

  # Apply to each review
  custom_labels = lr.predict(X_test_tfidf)

  # Store predictions
  pred_dict[f'slb_rv_{code}_custom'] = custom_labels
  custom_results[code+'_custom'] = {
      'y_pred': custom_labels,
      'compound_scores': all_probs.max(axis=1),
      'y_test': y_test,
      'X_test': X_test,
      'all_probs':all_probs,
      'vectorizer':vectorizer,
      'model_object': lr
  }

  cm_dict[var] = plot_confusion_matrix(custom_results[code+'_custom']['y_test'], custom_results[code+'_custom']['y_pred'], xticks=['Negative', 'Neutral', 'Positive'],yticks=['Negative', 'Neutral', 'Positive'], title= var + " - Confusion Matrix")

  custom_metrics = get_metrics_table(custom_results[code+'_custom']['y_test'], pred_dict)
  custom_metrics.sort_values(['accuracy'], ascending=False, inplace=True)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Class order: ['negative' 'neutral' 'positive']


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# Output results to excel for easier review

with pd.ExcelWriter(data_dir+"custom_models_results_test_"+td_date+".xlsx") as writer:
    data_split_summary.to_excel(writer, sheet_name = "train_test_split", index=False)
    custom_metrics.to_excel(writer, sheet_name = "metrics", index=False)

    workbook = writer.book
    worksheet = writer.sheets['metrics']

    row = 0

    # Add confusion matrices
    for plot_name, fig in cm_dict.items():
          # Set the figure size
          fig.set_size_inches(8, 6)
          # Save the figure to a temporary file
          img_path = plot_path + plot_name + '.png'
          fig.savefig(img_path, bbox_inches='tight')
          # Insert the image into the worksheet
          worksheet.insert_image(row, 10, img_path)
          plt.close(fig)  # Close the figure to free memory
          row += 30

writer.close()

/usr/local/lib/python3.12/dist-packages/xlsxwriter/workbook.py:404: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
# Save model object
with open(data_dir + f'tfid_lr_model_{td_date}.pkl', 'wb') as f:
  pickle.dump(custom_results['txt_tt_custom']['model_object'], f)

# Save vectorizer
with open(data_dir + f'tfid_lr_vectorizer_{td_date}.pkl', 'wb') as f:
  pickle.dump(custom_results['txt_tt_custom']['vectorizer'], f)

## Best model Results and mapping

### Run best model on the full data

Custom analyzer with TF-IDF + logistic Regression gave best results

In [ ]:
X_transformed = custom_results['txt_tt_custom']['vectorizer'].transform(reviews_df_clean['review_txt_tt'])

reviews_df_clean['slb_rv_txt_tt_custom'] = custom_results['txt_tt_custom']['model_object'].predict(X_transformed)
all_probs_custom = custom_results['txt_tt_custom']['model_object'].predict_proba(X_transformed)

# Save class probabilities
reviews_df_clean['ssc_neg_rv_txt_tt_custom'] = all_probs_custom[:,0]
reviews_df_clean['ssc_neu_rv_txt_tt_custom'] = all_probs_custom[:,1]
reviews_df_clean['ssc_pos_rv_txt_tt_custom'] = all_probs_custom[:,2]

# Max probability score for log reg = maximum probability
reviews_df_clean['ssc_rv_txt_tt_custom'] = all_probs_custom.max(axis=1)
reviews_df_clean

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,author_id,rating,is_recommended,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,helpfulness,total_feedback_count,total_pos_feedback_count,total_neg_feedback_count,unq_review_cnt_per_prod,review_txt_tt,sentiment_rating,slb_rv_txt_tt_custom,ssc_rv_txt_tt_custom
0,538863,1,0.0,2018-11-01,One use and into the trash this went. I woke u...,one and done,fair,blue,combination,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,NaN,0,0,0,10226,One use and into the trash this went. I woke u...,negative,positive,0.796214
1,561736,5,1.0,2018-07-28,This is my nightly hero. It is the one facial...,Awesome!,light,blue,combination,blonde,P421998,Midnight Recovery Concentrate Moisturizing Fac...,Kiehl's Since 1851,56.0,0.833333,6,5,1,729,This is my nightly hero. It is the one facial...,positive,positive,0.998441
2,561736,5,1.0,2018-07-28,This is my nightly hero. It is the one facial...,Awesome!,light,blue,combination,blonde,P445951,Midnight Recovery Concentrate Moisturizing Fac...,Kiehl's Since 1851,30.0,0.833333,6,5,1,729,This is my nightly hero. It is the one facial...,positive,positive,0.998441
3,602980,5,1.0,2018-10-03,This has become a must have addition to my mor...,Brain Must!!,lightMedium,blue,combination,blonde,P423159,Brain Dust,Moon Juice,38.0,1.000000,1,1,0,37,This has become a must have addition to my mor...,positive,positive,0.999235
4,696309,5,1.0,2019-06-17,"Well, I was a skeptic, but after trying a samp...",replacing my other serums,lightMedium,brown,dry,brown,P444222,Luxury Beauty Serum Calming Treatment,Saint Jane Beauty,125.0,1.000000,30,30,0,247,"Well, I was a skeptic, but after trying a samp...",positive,positive,0.864109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581780,orderGen270100,5,1.0,2019-02-28,I have dry sensitive combination skin and my f...,Can’t beat the price and you can see the resul...,fair,blue,combination,blonde,P427419,Hyaluronic Acid 2% + B5 Hydrating Serum,The Ordinary,15.7,NaN,0,0,0,2351,I have dry sensitive combination skin and my f...,positive,positive,0.946806
581781,orderGen39837,5,1.0,2018-05-18,This is great for fine lines! I’ve never revie...,Actually working on my fine lines!,light,green,normal,blonde,P429515,C-Tango Vitamin C Eye Cream,Drunk Elephant,64.0,0.250000,8,2,6,917,This is great for fine lines! I’ve never revie...,positive,positive,0.991425
581782,orderGen39837,5,1.0,2020-08-24,I got a facial and was told I have beautiful s...,Love this product,light,green,normal,blonde,P456418,Wild Rose Night-Brightening Sleeping Facial,KORRES,50.0,0.900000,10,9,1,222,I got a facial and was told I have beautiful s...,positive,positive,0.999607
581783,orderGen51156,5,1.0,2020-02-10,Nice cooling sensation once applied to the ski...,cooling and moisturizing,lightMedium,brown,combination,black,P433443,Aqua Bomb Sleeping Mask,belif,38.0,0.000000,1,0,1,627,Nice cooling sensation once applied to the ski...,positive,positive,0.992707


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Aggregate scores to product level

In [ ]:
# Calculate quality score (quick method)
reviews_df_clean['review_quality_score'] = (
    (reviews_df_clean['review_text'].str.len() / 500).clip(upper=1) * 0.4 +  # Length
    reviews_df_clean['ssc_rv_txt_tt_custom'] * 0.3 +                              # Confidence (if custom model)
    (reviews_df_clean['sentiment_rating'] == reviews_df_clean['slb_rv_txt_tt_custom']).astype(float) * 0.3  # Accuracy
)

# Aggregate to product level
reviews_prod_lvl = reviews_df_clean.groupby(['product_id','product_name']).agg({

    # Review counts
    'author_id': 'count',  # Total number of reviews

    # Sentiment distribution
    'sentiment_rating': [
        lambda x: (x == 'positive').sum(),   # Count positive
        lambda x: (x == 'neutral').sum(),    # Count neutral
        lambda x: (x == 'negative').sum(),   # Count negative
        lambda x: x.mode()[0] if len(x) > 0 else np.nan  # Most common sentiment
    ],

    # Model predictions
    'slb_rv_txt_tt_custom': lambda x: x.mode()[0] if len(x) > 0 else np.nan,  # Most common prediction

    'ssc_rv_txt_tt_custom': 'mean',

    # Quality metrics
    'review_quality_score': 'mean',  # Average quality score

    'rating': 'mean',  # Average star rating

}).reset_index()

# Flatten multi-level column names
reviews_prod_lvl.columns = [
    'product_id',
    'product_name',
    'total_reviews',
    'positive_rating_count',
    'neutral_rating_count',
    'negative_rating_count',
    'dominant_rating_sentiment',
    'predicted_sentiment',
    'predicted_sentiment_score',
    'avg_review_quality',
    'avg_rating'
]

# Add sentiment percentages
reviews_prod_lvl['positive_rating_pct'] = (
    reviews_prod_lvl['positive_rating_count'] / reviews_prod_lvl['total_reviews'] * 100
).round(2)

reviews_prod_lvl['neutral_rating_pct'] = (
    reviews_prod_lvl['neutral_rating_count'] / reviews_prod_lvl['total_reviews'] * 100
).round(2)

reviews_prod_lvl['negative_rating_pct'] = (
    reviews_prod_lvl['negative_rating_count'] / reviews_prod_lvl['total_reviews'] * 100
).round(2)


# Get top 5 reviews per product and format as numbered string
best_reviews_formatted = (
    reviews_df_clean
    .sort_values('review_quality_score', ascending=False)
    .groupby(['product_id'])
    .head(5)
    .groupby('product_id')['review_text']
    .apply(lambda reviews: ' ; '.join([f"{i+1}. {review}" for i, review in enumerate(reviews)]))
    .rename('review_sample')
).reset_index()

# Merge with product summary
reviews_prod_lvl = reviews_prod_lvl.merge(
    best_reviews_formatted,
    on=['product_id'],
    how='left'
)


final_columns = [
    'product_id',
    'product_name',
    'total_reviews',
    'avg_rating',
    'positive_rating_count',
    'neutral_rating_count',
    'negative_rating_count',
    'positive_rating_pct',
    'neutral_rating_pct',
    'negative_rating_pct',
    'dominant_rating_sentiment',
    'predicted_sentiment',
    'predicted_sentiment_score',
    'avg_review_quality',
    'review_sample'
]

reviews_prod_lvl = reviews_prod_lvl[final_columns]

# 8. Sort by total reviews (descending)
reviews_prod_lvl = reviews_prod_lvl.sort_values('total_reviews', ascending=False)

reviews_prod_lvl

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,product_id,product_name,total_reviews,avg_rating,positive_rating_count,neutral_rating_count,negative_rating_count,positive_rating_pct,neutral_rating_pct,negative_rating_pct,dominant_rating_sentiment,predicted_sentiment,predicted_sentiment_score,avg_review_quality,review_sample
387,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,10226,4.327303,8259,689,1278,80.76,6.74,12.50,positive,positive,0.930625,0.764323,"1. I have really dry and chapped lips, even mo..."
484,P427421,Protini Polypeptide Firming Refillable Moistur...,4075,3.964908,2846,423,806,69.84,10.38,19.78,positive,positive,0.915139,0.764656,1. this is hands down the best moisturizer i h...
931,P450271,Green Clean Makeup Meltaway Cleansing Balm Lim...,3887,4.494983,3350,185,352,86.18,4.76,9.06,positive,positive,0.940031,0.782722,1. This stuff is *amazing* - waterproof mascar...
359,P417238,Green Clean Makeup Removing Cleansing Balm,3887,4.494983,3350,185,352,86.18,4.76,9.06,positive,positive,0.940031,0.782722,1. This stuff is *amazing* - waterproof mascar...
60,P269122,Alpha Beta Extra Strength Daily Peel Pads,3804,4.597003,3524,147,133,92.64,3.86,3.50,positive,positive,0.946659,0.769754,1. Amazing product. I have acne prone skin wit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,P480176,Mini Calendula Deep Clean Foaming Face Wash,1,5.000000,1,0,0,100.00,0.00,0.00,positive,positive,0.995361,0.684208,1. Love this! Smells so good and a little goe...
1594,P475083,Skincare Essentials Kit,1,4.000000,1,0,0,100.00,0.00,0.00,positive,positive,0.999911,0.783173,1. I love this Clarins double serum! I’ve used...
1582,P474941,3 Step Intro Kit Type II,1,3.000000,0,1,0,0.00,100.00,0.00,neutral,neutral,0.999974,0.687192,1. Didn’t see much of a difference however it’...
2283,P504882,Bye Bye Bumps - Best of Body Kit,1,5.000000,1,0,0,100.00,0.00,0.00,positive,positive,0.999949,0.751185,1. I love this so much. Definitely worth the p...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Save data with sentiment scores

In [ ]:
reviews_prod_lvl.to_pickle(data_dir+f'reviews_prod_lvl_{td_date}.pkl')

## Ulta Reviews data

New reviews source: https://www.kaggle.com/datasets/nenamalikah/nlp-ulta-skincare-reviews/data

In [ ]:
# Import model object and vectorizer
import joblib

# Load sklearn objects
vectorizer = joblib.load(data_dir + 'tfid_lr_vectorizer_12_17_2025.pkl')
tfidf_lr_model = joblib.load(data_dir + 'tfid_lr_model_12_17_2025.pkl')

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nenamalikah/nlp-ulta-skincare-reviews")

print("Path to dataset files:", path)

In [ ]:
ulta_products_reviews = pd.read_csv(path+'/Ulta Skincare Reviews.csv')
ulta_products_reviews

In [ ]:
# Create reviews text + title filed
ulta_products_reviews['review_txt_tt'] = ulta_products_reviews['Review_Text'].astype(str) + " — " + ulta_products_reviews['Review_Title'].astype(str)
print(reviews_df_clean['review_txt_tt'].notnull().mean())

In [ ]:
# Score new data with saved model

X_ulta_transformed = vectorizer.transform(ulta_products_reviews['review_txt_tt'])

ulta_products_reviews['slb_rv_txt_tt_custom'] = tfidf_lr_model.predict(X_ulta_transformed)
all_probs_custom = tfidf_lr_model.predict_proba(X_ulta_transformed)

# Save class probabilities
ulta_products_reviews['ssc_neg_rv_txt_tt_custom'] = all_probs_custom[:,0]
ulta_products_reviews['ssc_neu_rv_txt_tt_custom'] = all_probs_custom[:,1]
ulta_products_reviews['ssc_pos_rv_txt_tt_custom'] = all_probs_custom[:,2]

# Max probability score for log reg = maximum probability
ulta_products_reviews['ssc_rv_txt_tt_custom'] = all_probs_custom.max(axis=1)
ulta_products_reviews

In [ ]:
# Save new dataset - No roll up, since this is smaller than our Sephora Reviews table
ulta_products_reviews.to_pickle(data_dir+ "ulta_reviews_scored.pkl")

## Code references

### Vader Sentiment Analyzer

Sentiment Analysis Article: https://medium.com/@rslavanyageetha/vader-a-comprehensive-guide-to-sentiment-analysis-in-python-c4f1868b0d2e

